# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


import required packages


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

helper functions to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


scraping the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### TASK 1: Requesting the Falcon9 Launch Wiki page from its URL


In [ ]:
# requests.get() method with the provided static_url
# assign the response to a object
data = requests.get(static_url).text

In [ ]:
# BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data, 'html5lib')

In [ ]:
# soup.title attribute
soup.title

### TASK 2: Extracting all column/variable names from the HTML table header


find all tables on the wiki page first

In [ ]:
# find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all("tr")


In [ ]:
first_launch_table = []

Starting from the third table is our target table contains the actual launch records.


In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


In [ ]:
print(first_launch_table.find_all("th"))

need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [ ]:
column_names = []
for row in first_launch_table.find_all("th"):
    name = extract_column_from_header(row)
    if((name != None) and (len(name)) > 0):
        column_names.append(extract_column_from_header(row))
# Applying find_all() function with `th` element on first_launch_table
# Iterating each th element and apply the provided extract_column_from_header() to get a column name
# Appending the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Checking the extracted column names


In [ ]:
print(column_names)

## TASK 3: Creating a data frame by parsing the launch HTML tables


Creating an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
launch_dict= dict.fromkeys(column_names)
# Remove an irrelvant column
del launch_dict['Date and time ( )']
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


In [ ]:
import numpy as np

In [ ]:
#df2 = pd.DataFrame(columns=['Flight No.','Date','Time','Version Booster','Launch_site','PayLoad','Payload mass','Orbit','Customer','Launch outcome','Booster landing'])
df3 = pd.DataFrame()

In [ ]:
df3.head()

In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    print('run ' ,table_number)
    print('size: ', np.size(table))
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #print(row)
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            print(flight_number)
            datatimelist=date_time(row[0])
            launch_dict['Flight No.'] = flight_number
            
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date
            print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'] = time
            print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
                launch_dict['Version Booster'] = bv
            print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'] = launch_site
            print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'] = payload
            print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            payload_mass = str(get_mass(row[4])).strip('~').strip('kg').strip(' ').replace(',','').replace('C','0').replace('5000–6000','5500')
            launch_dict['Payload mass'] = int(payload_mass)
            print(payload_mass)
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'] = orbit
            print(orbit)
            
            # Customer
            if((row[6].a) is None):
                customer = 'Various'
                print('Various')
            else:
                customer = row[6].a.string
                #launch_dict['Customer'] = customer
            launch_dict['Customer'] = customer
            print(customer)
            
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_outcome
            print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'] = booster_landing
            
            print(booster_landing)
            df3 = df3.append(launch_dict, ignore_index=True)
            

parsing launch record values into `launch_dict` to create a dataframe from it.


In [42]:
payload_mass_mean = df3['Payload_mass'].mean()

KeyError: 'Payload_mass'

In [41]:
df3['Payload mass'].replace(0, payload_mass_mean)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [38]:
df3

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success\n,[],Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0.0,LEO,NASA,Success,[],Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,[],No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,[],No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,[],No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
237,117,CCSFS,Starlink,15600,LEO,SpaceX,Success\n,F9 v1.1,Success,9 May 2021,06:42
238,118,KSC,Starlink,14000,LEO,SpaceX,Success\n,F9 v1.1,Success,15 May 2021,22:56
239,119,CCSFS,Starlink,15600,LEO,SpaceX,Success\n,F9 v1.1,Success,26 May 2021,18:59
240,120,KSC,SpaceX CRS-22,3328,LEO,NASA,Success\n,F9 v1.1,Success,3 June 2021,17:29


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [23]:
df3.to_csv('spacex_web_scraped.csv', index=False)